#Lab.11 / IBM3202 – Predicción de interacciones a partir del análisis coevolutivo de información de secuencias


###Aspectos teóricos

Como hemos discutido, las **secuencias naturales de proteínas pueden variar considerablemente** en términos de identidad de secuencia y, sin embargo, **conservar su estructura** y función. Un ejemplo llamativo de esto es la **familia de proteínas AAA de ATPasas** (Fig. 1), presente en todos los dominios de la vida. Todos los miembros de la familia tienen una casete AAA común responsable de la unión y hidrólisis de nucleótidos aunque **muestran una identidad de secuencia tan baja como el 20%**. Estos resultados demuestran que la conservación de la estructura y la función puede existir incluso en secuencias poco conservadas. Las mismas observaciones se pueden hacer para el análisis de **ARN no codificantes** que catalizan reacciones bioquímicas y se unen a ligandos específicos al **plegarse en estructuras tridimensionales complejas**, como en el caso de ribozimas y riboswitches.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/dca_01.png'/>
<figcaption>FIGURA 1. Arquitectura y conservación de secuencia de la casete AAA, con sus diferentes dominios mostrados como cajas de colores, ilustrando la baja identidad de secuencia de estas proteínas. Se detallan los patrones de secuencia típicos de la familia AAA. Además del código estándar de una letra para los aminoácidos, se utilizan los siguientes símbolos: <b>-</b> = D/E, <b>+</b> = R/K/H, <b>*</b> = cargado, <b>O</b> = grande e hidrofóbico. Los colores de las secuencias denotan la conservación: rojo, verde, negro significa casi absolutamente conservado, bien conservado y conservado, respectivamente.</figcaption>
 <br> Beyer A (1997) <i> Protein Sci 6(10), 2043-2058</i></figcaption></center>
</figure>

Una **explicación física** para esta aparente paradoja surge al considerar que las secuencias de proteínas y ARN no codificante son **fragmentos de información unidimensionales**, pero estas biomoléculas utilizan todas las **cuatro dimensiones** (3 espaciales y 1 temporal) **para definir su espacio conformacional**. Sin embargo, como vimos en nuestras simulaciones de plegamiento de proteínas, las proteínas no exploran la totalidad de su espacio conformacional posible (también conocido como la paradoja de Levinthal).

Para que estas biomoléculas exploren de manera eficiente y adecuada su espacio conformacional hacia su estructura nativa, se deben formar **contactos específicos (nativos) entre residuos**. En términos físicos, estos pueden ser enlaces de hidrógeno, puentes salinos, interacciones de Van der Waals, etc. En cambio, podemos simplificar la definición de una interacción entre residuos considerando una **"interacción"** simplemente como **dos residuos que tienen un efecto entre sí** y, por lo tanto, están **restringidos en la estructura final**.

Así, la adecuación de las secuencias de proteínas y ARN no codificante para plegarse en estructuras tridimensionales definidas no está controlada por las secuencias en sí, sino por su producto final: el espacio conformacional y **cómo evolucionan las secuencias al mantener estas interacciones estabilizadoras entre dos o más residuos en dicha secuencia**.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/dca_02.gif'/>
<figcaption>Representación esquemática de la información de contacto entre pares de residuos en estructuras tridimensionales que se puede extraer del análisis coevolutivo de secuencias de proteínas y ARN. Las cajas punteadas muestran cómo los pares de residuos que interactúan están condicionados a coevolucionar para mantener interacciones que son cruciales para la estabilidad de la estructura final.
Tomado de GREMLIN,</figcaption>
<i>gremlin.bakerlab.org/gremlin_faq.php</i></figcaption></center>
</figure>

Entonces, surge una pregunta simple: **¿podemos inferir estas interacciones solo a partir de la información de secuencia?** En el ejemplo anterior, tenemos un par de residuos que interactúan y ocupan posiciones específicas en un alineamiento de secuencias múltiples y pueden pertenecer a una sola cadena o a cadenas de proteínas. Como parte de la deriva aleatoria, puede haber ocurrido una sustitución perjudicial, que haya **perturbado parcial o completamente la interacción**, pero esto podría revertirse mediante una **mutación complementaria** que **restauraría la interacción** a través de una química diferente. Esto significa que, para un contacto que tenga lugar en el estado nativo de una proteína, los **residuos involucrados en tal interacción pueden variar en identidad** como cualquier otro, pero están **mutuamente restringidos para mantener identidades compatibles con dicho contacto**. En última instancia, esto significa que las **interacciones que están restringidas en la estructura final están codificadas dentro de la información de secuencia como residuos que coevolucionan**.


##Resumen Experimental

En este tutorial (que se basa en gran medida en el tutorial de Mehari B. Zerihun y se encuentra en [GitHub de pyDCA](https://github.com/KIT-MBS/pydca)), ejemplificaremos cómo podemos **inferir contactos nativos a partir del análisis coevolutivo de información de secuencia**. Para variar, trabajaremos en el análisis coevolutivo de secuencias de **ARN**, en lugar de proteínas.

Nos centraremos especialmente en comparar las interacciones nativas experimentales y predichas para el **adenosine deaminase (_add_) A-riboswitch**, uno de los riboswitches estructuralmente más simples y miembro de la familia Rfam de **riboswitches de purina**. Al unirse a la adenina, el dominio aptámero del riboswitch _add_ A cambia su patrón de plegado, lo que altera la conformación de su plataforma de expresión aguas abajo y activa la expresión génica.

Para el análisis coevolutivo de secuencias de ARN, utilizaremos el método de **Análisis de Acoplamiento Directo (DCA)**. Las puntuaciones de DCA proporcionan información cuantitativa sobre la existencia de contactos físicos en la estructura tridimensional de una biomolécula, con pares de puntuaciones más altas que predicen con precisión contactos nativos que se pueden observar en estructuras de proteínas y ARN resueltas experimentalmente.


#Parte 0. Descargar e instalar el software necesario

Antes de comenzar, **recuerda iniciar el entorno de ejecución alojado** en Google Colab.

Luego, debemos instalar varios programas para realizar este tutorial. Específicamente:
- **biopython** para la manipulación de archivos PDB
- **py3Dmol** para la visualización de la estructura de proteínas.
- **infernal** (INFERence of RNA ALignment) para buscar y alinear secuencias de ARN basadas en modelos de covarianza de secuencia y consenso de estructura secundaria de ARN
- **pyDCA** para predecir contactos nativos a través del análisis de acoplamiento directo.

**NOTA:** Para el alineamiento de secuencias de proteínas, la opción recomendada sería **HMMER**, que permite buscar y alinear secuencias de proteínas basadas en modelos ocultos de Markov (HMM) de conservación específica de la posición.

**ADVERTENCIA⚠️:** pyDCA requiere una versión más reciente de varios módulos de Python, por lo que se solicitará un **reinicio del entorno de ejecución**. ¡No lo aceptes!


1. Primero instalamos infernal

In [ ]:
!apt-get install infernal

2. También necesitamos python3.7 para que pyDCA se ejecute correctamente

In [ ]:
!apt-get install python3.7 python3-pip python3.7-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

3. También necesitaremos **biopython** e **ipykernel** para nuestro Python 3.7 con el fin de instalar **pydca**, mientras que también necesitamos **py3Dmol**, **pydca** y **biopython** simultáneamente para ejecutar en nuestro entorno interactivo de Python 3.10, todos están disponibles a través de `pip`.2. Finalmente, instalaremos py3Dmol, pydca y biopython simultáneamente, ya que todos están disponibles a través de `pip`.


In [ ]:
!pip install biopython ipykernel pydca
!python3.10 -m pip install biopython py3Dmol

#Parte I – Visualiza la estructura del A-riboswitch

3. También necesitaremos **biopython** e **ipykernel** para nuestro Python 3.7 con el fin de instalar **pydca**, mientras que también necesitamos **py3Dmol**, **pydca** y **biopython** simultáneamente para ejecutar en nuestro entorno interactivo de Python 3.10, todos están disponibles a través de `pip`.2. Finalmente, instalaremos py3Dmol, pydca y biopython simultáneamente, ya que todos están disponibles a través de `pip`.


In [ ]:
#Importando el PDB
import os
from Bio.PDB import *
pdbid = ['1y26']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

2. Visualiza la estructura utilizando py3Dmol.

**NOTA:** Te damos el pequeño ejercicio de indicar tú mismo el ID de la cadena para 1Y26.


In [ ]:
```python
import py3Dmol
# Primero asignamos py3Dmol.view como vista
view = py3Dmol.view()
# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer los archivos PDB
view.addModel(open('1y26.pdb', 'r').read(), 'pdb')
# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')
# Aquí establecemos el estilo de visualización y el color
view.setStyle({'chain': 'X'}, {'cartoon': {'colorscheme': 'nucleic'}})
# Puedes activar las etiquetas para cada residuo si lo deseas
view.addResLabels({'resi': '13-83'}, {'fontColor': 'black', 'fontOpacity': 1, 'showBackground': 'false'})
# Aquí centraremos la molécula para su visualización
view.zoomTo()
# Y finalmente visualizamos las estructuras con el comando a continuación
view.show()

**Pregunta❓:** Son todas las interacciones de A-Riboswitch tipo Watson-Crick?

# Parte II: Generar un alineamiento de múltiples secuencias (MSA) de todos los miembros conocidos de la familia de riboswitch de purina utilizando infernal


Como se indicó anteriormente, el _add_ A-riboswitch es miembro de la familia de **riboswitch de purina** en la [base de datos de **Rfam**](http://rfam.xfam.org), una colección de familias de ARN. La ventaja de Rfam es que contiene **alineamientos iniciales** (seed alignments), es decir, pequeños MSA basados en un subconjunto de secuencias conocidas que se utilizan para generar un **modelo de covarianza de consenso** de conservación de secuencia y estructura.

Estos modelos son luego utilizados por Rfam para encontrar **todos los posibles homólogos de ARN para una familia dada**, y luego todas estas secuencias para cada familia de ARN también son **depositadas en esta base de datos**.

Utilizaremos estos alineamientos iniciales y las secuencias de los miembros conocidos de la familia como entradas para generar un MSA basado en modelos de covarianza utilizando infernal. El MSA resultante será nuestra entrada para nuestro análisis coevolutivo, utilizando la secuencia de _add_ A-riboswitch como referencia para la predicción de contactos entre pares de residuos.


1. Descargaremos primero la secuencia de 1Y26 en formato FASTA utilizando biopython. Un pequeño cambio respecto a scripts anteriores es que biopython descarga la secuencia de interés en el alfabeto de ADN. Por lo tanto, para obtener nuestra secuencia en el alfabeto de ARN, crearemos nuestro propio registro con `SeqRecord` y utilizaremos `transcribe()` para convertir de ADN a ARN.

**NOTA:** Nuevamente, te proporcionamos el pequeño ejercicio de dejarte configurar el ID correcto de la secuencia de ARN que queremos descargar.


In [ ]:
import os
from pathlib import Path
from Bio import SeqIO, Entrez
from Bio.SeqRecord import SeqRecord

seqlist = ['1Y26_X']  # Letras mayúscula

for n in seqlist:
    # Configurando tu correo electrónico para poder usar Entrez
    Entrez.email = 'tu.correo@uc.cl'
    # Aquí, configuramos un manipulador temporal con nuestra secuencia descargada en formato fasta
    temp = Entrez.efetch(db="sequences", rettype="fasta", id=n)
    # Creamos un archivo fasta para escribir nuestra secuencia descargada
    # Que desafortunadamente se descarga como ADN
    seq_out = open("1Y26.fasta", 'w')
    # Leyendo la información de la secuencia como una cadena en formato fasta
    seq = SeqIO.read(temp, format="fasta")
    print("La secuencia se descarga como ADN:\n" + seq.seq)
    # Transcribiendo la secuencia, creando un nuevo registro de ARN y guardándolo
    rnaseq = seq.seq.transcribe()
    print("La convertimos de vuelta a ARN:\n" + rnaseq)
    rrec = SeqRecord(rnaseq, id=seq.id, description="reference")
    SeqIO.write(rrec, seq_out, "fasta")
    # Cerrando tanto el manipulador temporal como el archivo FASTA
    temp.close()
    seq_out.close()


### 3. A continuación, iremos al [**Rfam** database](http://rfam.xfam.org) y buscaremos la familia de ARN apropiada. Puedes hacer esto fácilmente ingresando _purine_ en la casilla de búsqueda **_Jump To_**. Esta familia está anotada como **Purine (RF00167)**.

   Una vez aquí, obtendremos dos archivos diferentes:
   - Ve al menú **_Sequences_** para **descargar todas las secuencias** en formato FASTA, que utilizaremos para generar un alineamiento múltiple de secuencias (MSA).
   - Ve al menú **_Alignment_** y **descarga el alineamiento inicial (seed alignment)**, que se utiliza para construir un modelo de covarianza empleado como perfil para el MSA. Debes descargarlo en formato Stockholm desde el menú **_Formatting options_**.

   Importa ambos archivos descargados arrastrándolos a la pestaña **Files** en la barra lateral izquierda de Google Colab.


3. Ahora utilizaremos el módulo `cmbuild` de infernal para generar nuestro modelo de covarianza basado en el alineamiento inicial de Rfam. Este modelo se almacenará como un archivo `.cm`.

In [ ]:
!cmbuild RF00167.cm RF00167.stockholm.txt

4. Luego, alinearemos todas las secuencias descargadas utilizando nuestro modelo de covarianza disponible como perfil de MSA a través del módulo `cmalign` de infernal. Ten en cuenta que estamos agregando nuestra secuencia de referencia al archivo FASTA para incluirla en nuestro MSA. El MSA resultante se almacenará como un archivo **FASTA alineado** llamado `RF00167_aligned.afa`.

Dado que no estamos interesados en deleciones/inserciones de secuencias con respecto al modelo de consenso, incluiremos columnas coincidentes en la alineación de salida con la opción `--matchonly`.


In [ ]:
!gunzip -d RF00167.fa.gz
!cat RF00167.fa 1Y26.fasta > RF00167_full.fasta
!cmalign -o RF00167_aligned.afa --outformat AFA --matchonly RF00167.cm RF00167_full.fasta

Hemos obtenido un MSA de todas las secuencias conocidas de la familia de riboswitch de purina (RF00167) de Rfam. Como en nuestros tutoriales anteriores, puedes visualizar esta alineación usando [Alignment Viewer 2.0](https://fast.alignmentviewer.org/).


# Parte III: Análisis coevolutivo de riboswitches de purina y precisión en la predicción de contactos entre pares de residuos.


Ahora, utilizaremos pyDCA, una implementación en Python de DCA, para predecir primero pares de residuos en contacto en la estructura tridimensional de riboswitches de purina basándonos únicamente en la información de la secuencia, y luego determinar la precisión de nuestra predicción al establecer el número de verdaderos positivos en comparación con los contactos observados en la estructura resuelta del riboswitch A de _add_.

2. En primer lugar, recortaremos nuestro archivo MSA basándonos en la longitud de la secuencia de referencia, que en este caso corresponde a la secuencia del aptámero del riboswitch A de _add_ que fue resuelta mediante cristalografía de rayos X. La MSA de inicio está definida en la ruta `rna_msa_file`, mientras que nuestra secuencia está definida en el archivo `rna_refseq_file`. Los datos MSA recortados se guardarán en un archivo de salida `MSA_RF00167_Trimmed.fa` en formato FASTA. El programa instalado se ejecuta de la siguiente manera:

```
pydca trim_by_refseq <biomolecule>  <alignment.fa>  <refseq_file.fa> --remove_all_gaps --verbose
```

 Esto comparará nuestra secuencia objetivo con todas las secuencias en la alineación. Luego, seleccionará la más cercana y recortará todas las posiciones de brechas. La MSA resultante se generará en una carpeta con el nombre del archivo con el prefijo "Trimmed".

 **NOTA**: Para recortar secuencias de proteínas, solo necesitamos cambiar biomolecule='rna' a biomolecule='protein' y proporcionar los archivos respectivos de MSA y secuencia de referencia.



2. Then, we will trim our MSA file based on the lenght of the reference sequence, which in this case corresponds to the sequence of the aptamer of _add_ A-riboswitch that was solved by X-ray crystallography. The starting MSA is defined in the `rna_msa_file` path, whereas our sequence is defined in the `rna_refseq_file`. The trimmed MSA data to an output file `MSA_RF00167_Trimmed.fa` in FASTA format.

**NOTE**: To trim protein sequences, we just need to change biomolecule='rna' to biomolecule='protein' and provide the respective MSA and reference sequence files.

In [ ]:
!pydca trim_by_refseq rna RF00167_aligned.afa  1Y26.fasta --remove_all_gaps --verbose

Podemos ahora revisar que la primera secuencia esté correctamente recortada

In [ ]:
!head Trimmed_RF00167_aligned/Trimmed_RF00167_aligned.fa

3. Una vez hecho esto, calcularemos nuestros puntajes DCA utilizando un algoritmo de **maximización de pseudo-verosimilitud (plm)**. Esto difiere del algoritmo de **campo medio (mf)** que revisamos durante las clases. El análisis de este algoritmo está fuera del alcance de este tutorial y se describe de manera elegante [en el BiorXiv de pyDCA](https://www.biorxiv.org/content/10.1101/805523v1).

   Para este análisis, primero creamos una instancia de plmDCA llamada `plmdca_inst` para analizar nuestra MSA de RNA recortada. También proporcionamos una serie de parámetros opcionales. Las interacciones de 2 cuerpos $J_{ij}$ y los campos de sitios individuales $h_i$ y $h_j$ para cada posición en la MSA se estiman maximizando su pseudo-verosimilitud desde valores iniciales a través de un descenso de gradiente mediante un enfoque iterativo. El número de iteraciones se define con `max_iterations`. Además, se establece un límite máximo del 80% de identidad de secuencia para todas las secuencias en la MSA utilizando el parámetro `seqid`. Por último, el número de hilos se establece en 2, el máximo para Google Colab.

   Después de obtener los campos y las interacciones, los puntajes DCA se calculan a partir de la Información Directa (DI):
   <figure>
   <center>
   <img src='https://github.com/pb3lab/ibm3202/raw/master/images/dca_04.png'/>
   </center>
   </figure>

   Aquí llamaremos a la función plmdca con la función compute_fn, que calculará la norma de Frobenius del plmDCA, el Producto Promedio Corregido (APC) como vimos en las clases.


In [ ]:
!plmdca compute_fn rna Trimmed_RF00167_aligned/Trimmed_RF00167_aligned.fa --apc --seqid 0.8 --num_threads 2 --max_iterations 500

4. Veamos los primeros 10 contactos de a pares y sus puntajes DCA correspondientes.

In [ ]:
!head -n 22 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt

5. Una vez que hayamos obtenido nuestros puntajes DCA, utilizaremos el visualizador de DCA. Este visualizador toma el tipo de biomolécula (`'rna'`), nuestro archivo de secuencia de referencia (`refseq_file`) y nuestra lista previamente generada de puntajes DCA ordenados (`sorted_dca_scores`) como entrada para determinar sus pares de residuos en contacto basándose únicamente en la información de la secuencia.

   Cuando la información estructural está disponible, también toma un ID de PDB (`'1y26'`) y una cadena (`'x'`) como parámetros de entrada, así como distancias lineales especificadas (es decir, separación de secuencia entre pares de residuos, `linear_dist`) y distancias de contacto (`contact_dist`, en Å). Este visualizador toma las siguientes opciones:

```
pydca plot_contact_map <biomolecule> <PDB_chain_name> <PDB_id/PDB_file.PDB> <refseq.fa> <DCA_file.txt> --verbose  
```

**NOTA:** Cuando ejecutamos la celda anterior, vemos una advertencia `You didn't supply RNA secondary structure file`. Dado que proporcionar un archivo de estructura secundaria de ARN es opcional, podemos ignorar la advertencia.


In [ ]:
!pydca plot_contact_map --help

In [ ]:
!pydca plot_contact_map rna X 1Y26 1Y26.fasta\
 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt --verbose

6. Ahora graficaremos ambos mapas de contactos, con los contactos basados en la estructura en el triángulo superior izquierdo y los contactos basados en DCA en el triángulo inferior derecho. Los contactos predichos correctamente se muestran en verde, mientras que los falsos positivos se muestran en rojo. También note que el número de contactos DCA es equivalente a $L$, donde $L$ es la longitud (es decir, el número de columnas) de la alineación recortada.


In [ ]:
!awk 'NR>=23{print $5, $6, $1}' /content/contact_map_1Y26/contact_map1Y26.txt > contacts.txt
!echo X Y Z > /content/headers
!cat /content/headers /content/contacts.txt > /content/contacts2.txt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

data = np.genfromtxt('/content/contacts2.txt', dtype=None, names=True, encoding=None)
x = data['X']
y = data['Y']
z = data['Z']

# Define un diccionario para colorear
category_colors = {
    'tp': 'green',
    'fp': 'red',
    'pdb': 'gray'
}

# Crea listas de las coordenadas invertidas
inverted_x = []
inverted_y = []
colors = []

plt.figure(figsize=(6, 6))

for i in range(len(z)):
    category = z[i]
    if category in ('tp', 'fp'):
        inverted_x.append(y[i])  # Invierte tp y fp
        inverted_y.append(x[i])
        colors.append(category_colors[category])
    else:
        inverted_x.append(x[i])
        inverted_y.append(y[i])
        colors.append(category_colors[category])

plt.scatter(inverted_x, inverted_y, c=colors, marker='o', s=5)  # Puedes cambiar el tamaño 's' acorde
for category, color in category_colors.items():
    plt.scatter([], [], c=color, label=category)

plt.xlabel('Residue index')
plt.ylabel('Residue index')
plt.title('Contact map for the mfDCA prediction')
plt.legend(bbox_to_anchor=(1.05, 1.05))
plt.grid(True)
plt.show()

7. Por último, determinaremos la precisión de nuestro análisis de coevolución al observar su tasa de verdaderos positivos (TP) por rango. La tasa de verdaderos positivos por rango es el número de contactos correctamente predichos por rango de los pares predichos dividido por todas las predicciones en ese rango.

In [ ]:
!pydca plot_tp_rate rna X 1Y26 1Y26.fasta\
 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt --verbose

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
data = pd.read_csv('/content/TPR_1Y26/TPR_1Y26.txt', delimiter='\t', skiprows=11, header=None, usecols=[0])

data['Index'] = range(1, len(data) + 1)
data.set_index('Index', inplace=True)
plt.semilogx(data.index, data[0], linestyle='-')
plt.xlabel("Rank")
plt.ylabel("TPR")
plt.title("True positive rate for sorted APC")
plt.show()

En este gráfico resultante, la línea azul corresponde a las tasas de verdaderos positivos para los contactos predichos, y la línea naranja corresponde a la tasa de verdaderos positivos teóricamente máxima posible para los contactos obtenidos de la estructura PDB.

**¡Esto marca el final del undécimo tutorial!** ¡Buena ciencia!